In [1]:
import pandas as pd


ratings_df = pd.read_csv("data/Ratings.csv", sep=";")
books_df = pd.read_csv("data/Books.csv", sep=";")
users_df = pd.read_csv("data/Users.csv", sep=";")

# Заменяем нечисловые значения на NaN и конвертируем в Int32
users_df["User-ID"] = pd.to_numeric(users_df["User-ID"], errors="coerce").astype(
    "Int32"
)
users_df["Age"] = pd.to_numeric(users_df["Age"], errors="coerce").astype("Int32")

/var/folders/4p/dl7135912qnd6kpg_p9nlm7h0000gn/T/ipykernel_14600/3705107979.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('data/Users.csv', sep=';')


In [10]:
ratings_df.head()

,User-ID,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [11]:
books_df.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [12]:
users_df.head()

,User-ID,Age
0,1,<NA>
1,2,18
2,3,<NA>
3,4,17
4,5,<NA>


## Подготовка данных для обучения модели

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time
import numpy as np

from lfm import LFM

df = pd.merge(pd.merge(ratings_df, books_df, on="ISBN"), users_df, on="User-ID")
df = df[df["Rating"] != 0]

print(f"Размер данных: {df.shape}")
print(f"Количество уникальных пользователей: {df['User-ID'].nunique()}")
print(f"Количество уникальных книг: {df['ISBN'].nunique()}")
print(f"Диапазон рейтингов: {df['Rating'].min()} - {df['Rating'].max()}")

X = df[["User-ID", "ISBN"]]
y = df["Rating"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Размер тренировочной выборки: {X_train.shape[0]}")
print(f"Размер тестовой выборки: {X_test.shape[0]}")

/Users/alexanderivanov/programs/python/spring-25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Размер данных: (383856, 8)
Количество уникальных пользователей: 68092
Количество уникальных книг: 149842
Диапазон рейтингов: 1 - 10
Размер тренировочной выборки: 307084
Размер тестовой выборки: 76772


## Обучение собственной модели LFM


In [6]:
lfm = LFM(
    n_factors=50,
    n_epochs=20,
    lr=0.01,
    reg=0.02,
    random_state=42,
    verbose=True,
)

start_time = time.time()
lfm.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Время обучения LFM: {training_time:.2f} сек")

Обучение LFM: 100%|██████████| 20/20 [02:12<00:00,  6.61s/it]

Время обучения LFM: 132.51 сек


## Оценка качества и времени предсказания


In [7]:
start_time = time.time()
y_pred = lfm.predict(X_test)
prediction_time = time.time() - start_time

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE на тестовой выборке: {rmse:.4f}")
print(f"Время предсказания LFM: {prediction_time:.4f} сек")

print(f"Средний абсолютный отклонение (MAE): {np.mean(np.abs(y_test - y_pred)):.4f}")
print(f"Среднее предсказанное значение: {np.mean(y_pred):.4f}")
print(f"Среднее истинное значение: {np.mean(y_test):.4f}")

RMSE на тестовой выборке: 1.6163
Время предсказания LFM: 1.1022 сек
Средний абсолютный отклонение (MAE): 1.2415
Среднее предсказанное значение: 7.6835
Среднее истинное значение: 7.6287


## Сравнение с эталонной реализацией из Surprise


In [3]:
from surprise import SVD, Dataset, Reader

print("Подготовка данных для Surprise...")

# Подготавливаем данные в формате для Surprise
train_data = pd.merge(
    X_train, y_train.to_frame("Rating"), left_index=True, right_index=True
)

# Создаем Reader для Surprise
reader = Reader(rating_scale=(1, 10))

# Загружаем тренировочные данные
surprise_dataset = Dataset.load_from_df(
    train_data[["User-ID", "ISBN", "Rating"]], reader
)
surprise_trainset = surprise_dataset.build_full_trainset()

# Создаем алгоритм SVD с аналогичными параметрами
surprise_svd = SVD(
    n_factors=50, n_epochs=20, lr_all=0.01, reg_all=0.02, random_state=42
)

print("Обучение Surprise SVD...")
start_time = time.time()
surprise_svd.fit(surprise_trainset)
surprise_training_time = time.time() - start_time

print(f"Время обучения Surprise SVD: {surprise_training_time:.2f} сек")

Подготовка данных для Surprise...
Обучение Surprise SVD...
Время обучения Surprise SVD: 2.98 сек


In [4]:
# Предсказание для Surprise SVD
print("Предсказание Surprise SVD...")
start_time = time.time()

# Получаем предсказания для тестовой выборки
y_pred_surprise = []
for _, row in X_test.iterrows():
    user_id, isbn = row["User-ID"], row["ISBN"]

    # Делаем предсказание
    prediction = surprise_svd.predict(user_id, isbn)
    y_pred_surprise.append(prediction.est)

y_pred_surprise = np.array(y_pred_surprise)
surprise_prediction_time = time.time() - start_time

# Вычисляем метрики для Surprise SVD
rmse_surprise = np.sqrt(mean_squared_error(y_test, y_pred_surprise))

print(f"RMSE (Surprise SVD): {rmse_surprise:.4f}")
print(f"Время предсказания Surprise SVD: {surprise_prediction_time:.4f} сек")
print(f"MAE (Surprise SVD): {np.mean(np.abs(y_test - y_pred_surprise)):.4f}")

Предсказание Surprise SVD...
RMSE (Surprise SVD): 1.6406
Время предсказания Surprise SVD: 1.2769 сек
MAE (Surprise SVD): 1.2613


## Сравнительная таблица результатов


In [8]:
# Создаем сравнительную таблицу
results_df = pd.DataFrame(
    {
        "Модель": ["Собственная LFM", "Surprise SVD"],
        "RMSE": [rmse, rmse_surprise],
        "MAE": [
            np.mean(np.abs(y_test - y_pred)),
            np.mean(np.abs(y_test - y_pred_surprise)),
        ],
        "Время обучения (сек)": [training_time, surprise_training_time],
        "Время предсказания (сек)": [prediction_time, surprise_prediction_time],
    }
)

print("Сравнение моделей:")
print(results_df.to_string(index=False))

# Выводы
print("\n" + "=" * 50)
print("ВЫВОДЫ:")
print("=" * 50)

if rmse < rmse_surprise:
    print("✅ Собственная LFM показала лучшее качество по RMSE")
else:
    print("❌ Surprise SVD показала лучшее качество по RMSE")

if training_time < surprise_training_time:
    print("✅ Собственная LFM обучается быстрее")
else:
    print("❌ Surprise SVD обучается быстрее")

print(f"Разница в RMSE: {abs(rmse - rmse_surprise):.4f}")
print(
    f"Разница во времени обучения: {abs(training_time - surprise_training_time):.2f} сек"
)

Сравнение моделей:
         Модель     RMSE      MAE  Время обучения (сек)  Время предсказания (сек)
Собственная LFM 1.616271 1.241472            132.505874                  1.102199
   Surprise SVD 1.640645 1.261276              2.977292                  1.276916

ВЫВОДЫ:
✅ Собственная LFM показала лучшее качество по RMSE
❌ Surprise SVD обучается быстрее
Разница в RMSE: 0.0244
Разница во времени обучения: 129.53 сек
